In [1]:
'''
PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group 
the passenger is travelling with and pp is their number within the group. People in a group are often family members, 
but not always.

HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.

CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the 
voyage. Passengers in cryosleep are confined to their cabins.

Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P 
for Port or S for Starboard.

Destination - The planet the passenger will be debarking to.

Age - The age of the passenger.

VIP - Whether the passenger has paid for special VIP service during the voyage.

RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship 
Titanic's many luxury amenities.

Name - The first and last names of the passenger.

Transported - Whether the passenger was transported to another dimension. This is the target, the column you are 
trying to predict.
'''

"\nPassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group \nthe passenger is travelling with and pp is their number within the group. People in a group are often family members, \nbut not always.\n\nHomePlanet - The planet the passenger departed from, typically their planet of permanent residence.\n\nCryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the \nvoyage. Passengers in cryosleep are confined to their cabins.\n\nCabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P \nfor Port or S for Starboard.\n\nDestination - The planet the passenger will be debarking to.\n\nAge - The age of the passenger.\n\nVIP - Whether the passenger has paid for special VIP service during the voyage.\n\nRoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship \nTitanic's many luxury amenit

In [2]:
# Import Dependencies for Download and Clean
# run in terminal to use bigquery  "pip install --upgrade google-cloud-bigquery"
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
'/Users/spicious/Desktop/News Datasets/Resources/spaceship-titanic-387720-729aac731f9f.json')

In [3]:
# Import Dependencies for Random Forest Model
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [4]:
# Import for Data Imputation
from scipy import stats
import numpy as np
from sklearn import impute

In [5]:
project_id = 'spaceship-titanic-387720'
client = bigquery.Client(credentials= credentials,project=project_id)

In [6]:
query_Amenities = client.query("""
   SELECT *
   FROM Starship_Titanic.Amenities""")

query_PassengerInfo = client.query("""
   SELECT *
   FROM Starship_Titanic.PassengerInfo""")

query_PlanetInfo = client.query("""
   SELECT *
   FROM Starship_Titanic.Planet""")

results_Amenities = query_Amenities.result()
results_PassengerInfo = query_PassengerInfo.result()
results_PlanetInfo = query_PlanetInfo.result()

Amenities_df = pd.DataFrame(results_Amenities)
PassengerInfo_df = pd.DataFrame(results_PassengerInfo)
PlanetInfo_df = pd.DataFrame(results_PlanetInfo)

In [7]:
print(Amenities_df.loc[0][0])
print(PassengerInfo_df.loc[0][0])
print(PlanetInfo_df.loc[0][0])

Row(('4446_05', 'B/175/S', False, 0.0, 4017.0, None, None, 2260.0), {'PassengerId': 0, 'Cabin': 1, 'VIP': 2, 'RoomService': 3, 'FoodCourt': 4, 'ShoppingMall': 5, 'Spa': 6, 'VRDeck': 7})
Row(('6512_02', 'Photons Drivery', None, 'A/80/S', None, True), {'PassengerId': 0, 'Name': 1, 'HomePlanet': 2, 'Cabin': 3, 'Age': 4, 'Transported': 5})
Row(('3896_01', None, '55 Cancri e', None), {'PassengerId': 0, 'HomePlanet': 1, 'Destination': 2, 'CryoSleep': 3})


In [8]:
Amenities_sorted = pd.DataFrame()
i = 0
while i < 8:
    Amenities_sorted[i] = Amenities_df[0].apply(lambda x: x[i])
    i += 1

Amenities_sorted = Amenities_sorted.rename(columns={0: "PassengerId", 
                   1: "Cabin", 
                   2: "VIP", 
                   3: "RoomService",
                   4: "FoodCourt",
                   5: "ShoppingMall",
                   6: "Spa",
                   7: "VRDeck"})

In [9]:
Amenities_sorted

,PassengerId,Cabin,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,4446_05,B/175/S,False,0.0,4017.0,NaN,NaN,2260.0
1,8906_01,F/1723/S,False,720.0,2.0,NaN,NaN,0.0
2,2122_01,E/154/S,False,0.0,618.0,NaN,NaN,0.0
3,3008_01,D/92/P,False,0.0,0.0,NaN,NaN,0.0
4,0008_02,B/1/P,False,0.0,0.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
12965,2289_01,F/465/P,False,20.0,0.0,2559.0,20.0,0.0
12966,3567_01,F/739/P,False,0.0,0.0,2687.0,35.0,340.0
12967,4012_01,B/153/S,False,0.0,7000.0,8251.0,1523.0,0.0
12968,7941_01,C/257/P,False,0.0,116.0,10705.0,9181.0,10.0


In [10]:
PassengerInfo_sorted = pd.DataFrame()
i = 0
while i < 6:
    PassengerInfo_sorted[i] = PassengerInfo_df[0].apply(lambda x: x[i])
    i += 1
    
PassengerInfo_sorted = PassengerInfo_sorted.rename(columns={0: "PassengerId", 
                   1: "Name", 
                   2: "HomePlanet", 
                   3: "Cabin",
                   4: "Age",
                   5: "Transported"})

In [11]:
PassengerInfo_sorted

,PassengerId,Name,HomePlanet,Cabin,Age,Transported
0,6512_02,Photons Drivery,None,A/80/S,NaN,True
1,8202_03,Charga Unkcatted,None,C/306/S,NaN,True
2,1050_01,Evendy Buchansen,None,G/157/P,NaN,None
3,6081_01,Ecco Shpie,None,F/1256/P,NaN,None
4,6550_01,Grumak Porkilte,None,B/252/S,NaN,None
...,...,...,...,...,...,...
12965,0818_01,Ksorab Couseced,Europa,B/29/S,71.0,True
12966,1502_04,Hadaton Donotaked,Europa,C/54/P,71.0,None
12967,1131_01,Mizark Comessic,Europa,B/44/S,75.0,None
12968,0378_03,Naslon Pirejus,Europa,B/11/S,79.0,False


In [12]:
PlanetInfo_sorted = pd.DataFrame()
i = 0
while i < 4:
    PlanetInfo_sorted[i] = PlanetInfo_df[0].apply(lambda x: x[i])
    i += 1
    
PlanetInfo_sorted = PlanetInfo_sorted.rename(columns={0: "PassengerId", 
                   1: "HomePlanet", 
                   2: "Destination", 
                   3: "CryoSleep"})

In [13]:
PlanetInfo_sorted

,PassengerId,HomePlanet,Destination,CryoSleep
0,3896_01,None,55 Cancri e,None
1,7711_01,None,TRAPPIST-1e,None
2,4420_01,None,TRAPPIST-1e,None
3,4984_02,None,PSO J318.5-22,None
4,8435_01,None,TRAPPIST-1e,None
...,...,...,...,...
12965,9166_01,Europa,55 Cancri e,True
12966,9177_01,Europa,55 Cancri e,True
12967,9177_02,Europa,55 Cancri e,True
12968,9215_01,Europa,TRAPPIST-1e,True


In [14]:
# Merging tables
merge_df = pd.merge(Amenities_sorted, PassengerInfo_sorted, on = 'PassengerId', how = 'inner')
df = pd.merge(merge_df, PlanetInfo_sorted, on = 'PassengerId', how = 'inner')

In [15]:
# Cleaning up Column Duplications
df = df.drop(['Cabin_y', 'HomePlanet_y'], axis=1)

In [16]:
# Renaming Columns
df = df.rename(columns={'Cabin_x': "Cabin", 
                   'HomePlanet_x': "HomePlanet"})

In [17]:
df["Transported"].value_counts()

True     4378
False    4315
Name: Transported, dtype: int64

In [18]:
# Drop all rows with missing null's in the training data column
df.dropna(subset = ['Transported'], inplace=True)

In [19]:
#check dataframe
print(df.shape)
df.head(5)

(8693, 14)


,PassengerId,Cabin,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,HomePlanet,Age,Transported,Destination,CryoSleep
0,4446_05,B/175/S,False,0.0,4017.0,NaN,NaN,2260.0,Phah Chocaters,Europa,33.0,True,TRAPPIST-1e,None
1,8906_01,F/1723/S,False,720.0,2.0,NaN,NaN,0.0,Rena Gainney,Earth,19.0,False,55 Cancri e,False
4,0008_02,B/1/P,False,0.0,0.0,NaN,0.0,0.0,Altardr Flatic,Europa,34.0,True,TRAPPIST-1e,True
5,0014_01,F/3/P,False,1286.0,122.0,NaN,0.0,0.0,Flats Eccle,Mars,27.0,False,55 Cancri e,False
6,0020_03,E/0/S,False,0.0,0.0,NaN,0.0,0.0,Mollen Mcfaddennon,Earth,29.0,False,55 Cancri e,True


In [20]:
# Check columns
columns = list(df.columns)
columns

['PassengerId',
 'Cabin',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name',
 'HomePlanet',
 'Age',
 'Transported',
 'Destination',
 'CryoSleep']

In [21]:
df.isnull().sum()

PassengerId       0
Cabin           199
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
HomePlanet      201
Age             179
Transported       0
Destination     182
CryoSleep       217
dtype: int64

# Data Munging

In [22]:
# Make plans
'''
1) Check for missing values in all columns
1.5) Consider Imputation
2) remove unnecessary columns such as PassengerId and Name
3) Convert categorical variables into indicator variables for HomePlanet, CryoSleep, Deck, Side, Destination, VIP, Transported
4) Split up Cabin column into three different features
'''

'\n1) Check for missing values in all columns\n1.5) Consider Imputation\n2) remove unnecessary columns such as PassengerId and Name\n3) Convert categorical variables into indicator variables for HomePlanet, CryoSleep, Deck, Side, Destination, VIP, Transported\n4) Split up Cabin column into three different features\n'

In [23]:
# Split Cabin column in three different Columns
df[['Deck','RoomNum','Side']] = df.Cabin.str.split("/", expand = True)
del df['Cabin']

# Reorder dataset to make me happy
df = df[['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Deck',
 'RoomNum',
 'Side',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name',
 'Transported']]

# Check to see everything is going to plan
print(df.head())

  PassengerId HomePlanet CryoSleep Deck RoomNum Side  Destination   Age  \
0     4446_05     Europa      None    B     175    S  TRAPPIST-1e  33.0   
1     8906_01      Earth     False    F    1723    S  55 Cancri e  19.0   
4     0008_02     Europa      True    B       1    P  TRAPPIST-1e  34.0   
5     0014_01       Mars     False    F       3    P  55 Cancri e  27.0   
6     0020_03      Earth      True    E       0    S  55 Cancri e  29.0   

     VIP  RoomService  FoodCourt  ShoppingMall  Spa  VRDeck  \
0  False          0.0     4017.0           NaN  NaN  2260.0   
1  False        720.0        2.0           NaN  NaN     0.0   
4  False          0.0        0.0           NaN  0.0     0.0   
5  False       1286.0      122.0           NaN  0.0     0.0   
6  False          0.0        0.0           NaN  0.0     0.0   

                 Name Transported  
0      Phah Chocaters        True  
1        Rena Gainney       False  
4      Altardr Flatic        True  
5         Flats Eccle     

In [24]:
# Drop unnecessary columns
del df['PassengerId']
del df['Name']

In [25]:
# Check drops
columns = list(df.columns)
columns

['HomePlanet',
 'CryoSleep',
 'Deck',
 'RoomNum',
 'Side',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Transported']

In [26]:
# examining missing values
print("Missing values distribution: ")
print(df.isnull().mean())

Missing values distribution: 
HomePlanet      0.023122
CryoSleep       0.024963
Deck            0.022892
RoomNum         0.022892
Side            0.022892
Destination     0.020936
Age             0.020591
VIP             0.023352
RoomService     0.020821
FoodCourt       0.021051
ShoppingMall    0.023927
Spa             0.021051
VRDeck          0.021627
Transported     0.000000
dtype: float64


In [27]:
# check datatype in each column
print("Column datatypes: ")
print(df.dtypes)

Column datatypes: 
HomePlanet       object
CryoSleep        object
Deck             object
RoomNum          object
Side             object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported      object
dtype: object


In [28]:
#df2['Transported'] = df2['Transported'].map({'True': True, 'False': False}) 
df['Transported'] = df['Transported'].astype(dtype = 'bool', errors = 'ignore')

In [29]:
# Exploring the data
print('HomePlanet', df["HomePlanet"].unique())
print('CryoSleep', df["CryoSleep"].unique())
print('Deck', df["Deck"].unique())
print('Side', df["Side"].unique())
print('Destination', df["Destination"].unique())
print('VIP', df["VIP"].unique())
print('Transported', df["Transported"].unique())

HomePlanet ['Europa' 'Earth' 'Mars' None]
CryoSleep [None False True]
Deck ['B' 'F' 'E' 'C' 'G' 'D' None 'A' 'T']
Side ['S' 'P' None]
Destination ['TRAPPIST-1e' '55 Cancri e' 'PSO J318.5-22' None]
VIP [False True None]
Transported [ True False]


In [30]:
df.isnull().sum()

HomePlanet      201
CryoSleep       217
Deck            199
RoomNum         199
Side            199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

In [31]:
df.shape

(8693, 14)

In [32]:
df["Transported"].value_counts()

True     4378
False    4315
Name: Transported, dtype: int64

# Nan Values

In [33]:
df_clean = df.dropna()
df_clean

,HomePlanet,CryoSleep,Deck,RoomNum,Side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
495,Europa,False,B,0,P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
497,Europa,True,B,1,P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,True
498,Earth,False,F,4,P,55 Cancri e,24.0,False,0.0,1.0,0.0,0.0,637.0,False
499,Mars,True,F,5,P,TRAPPIST-1e,45.0,False,0.0,0.0,0.0,0.0,0.0,True
500,Earth,False,G,0,P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12964,Europa,False,E,315,P,TRAPPIST-1e,32.0,False,8151.0,2052.0,1919.0,7352.0,754.0,False
12965,Mars,False,F,465,P,TRAPPIST-1e,37.0,False,20.0,0.0,2559.0,20.0,0.0,True
12966,Earth,False,F,739,P,TRAPPIST-1e,20.0,False,0.0,0.0,2687.0,35.0,340.0,False
12968,Europa,False,C,257,P,TRAPPIST-1e,41.0,False,0.0,116.0,10705.0,9181.0,10.0,False


In [34]:
compression_opts = dict(method='zip',
                        archive_name='titanic.csv')  
df.to_csv('titanic.zip', index=False,
          compression=compression_opts)  

# Random Forest Model

In [35]:
# Define features set
X = df_clean.copy()
X.drop("Transported", axis=1, inplace=True)
X.head()

,HomePlanet,CryoSleep,Deck,RoomNum,Side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
495,Europa,False,B,0,P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
497,Europa,True,B,1,P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0
498,Earth,False,F,4,P,55 Cancri e,24.0,False,0.0,1.0,0.0,0.0,637.0
499,Mars,True,F,5,P,TRAPPIST-1e,45.0,False,0.0,0.0,0.0,0.0,0.0
500,Earth,False,G,0,P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0


In [36]:
# Define target vector
y = df_clean["Transported"].values.reshape(-1, 1)
y[:5]

array([[False],
       [ True],
       [False],
       [ True],
       [ True]])

In [37]:
# Dummy Categorical Variables
X = pd.get_dummies(X)

In [38]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [39]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [40]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [41]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [43]:
# Fit the model and use .ravel()on the "y_train" data. 
rf_model = rf_model.fit(X_train, y_train)

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [44]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test)

In [45]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [46]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,705,137
Actual 1,177,672


Accuracy Score : 0.8143110585452396
Classification Report
              precision    recall  f1-score   support

       False       0.80      0.84      0.82       842
        True       0.83      0.79      0.81       849

    accuracy                           0.81      1691
   macro avg       0.81      0.81      0.81      1691
weighted avg       0.82      0.81      0.81      1691



In [47]:
# Get the feature importance array
importances = rf_model.feature_importances_
# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:100]

[(0.07447629146927538, 'Spa'),
 (0.07038660233216101, 'Age'),
 (0.06627293229783245, 'VRDeck'),
 (0.060892135966090576, 'RoomService'),
 (0.05977782598370213, 'CryoSleep_False'),
 (0.0593638955790175, 'FoodCourt'),
 (0.05354727770774385, 'ShoppingMall'),
 (0.045041322255381204, 'CryoSleep_True'),
 (0.01615520770857291, 'HomePlanet_Earth'),
 (0.014692994883978534, 'HomePlanet_Europa'),
 (0.0113692009082334, 'Deck_F'),
 (0.010792861543748868, 'Deck_E'),
 (0.009609066587193762, 'HomePlanet_Mars'),
 (0.009291254255882486, 'Deck_G'),
 (0.008925864084896505, 'Destination_TRAPPIST-1e'),
 (0.008747141443948593, 'Side_S'),
 (0.008394352650640834, 'Destination_55 Cancri e'),
 (0.007896829754068423, 'Side_P'),
 (0.0060339625016005505, 'Deck_B'),
 (0.005696585020079958, 'Deck_C'),
 (0.004452545596153306, 'Destination_PSO J318.5-22'),
 (0.0031387608270332097, 'Deck_D'),
 (0.0018817413995296873, 'Deck_A'),
 (0.0018027751762501503, 'VIP_False'),
 (0.001687493209203072, 'VIP_True'),
 (0.00162536957757

# Trying Data Imputation

In [48]:
# Impute int values with mean.

titanic_df_impute = df.copy()

mean_imputer = impute.SimpleImputer(strategy='mean')

titanic_df_impute['Age'] = mean_imputer.fit_transform(titanic_df_impute['Age'].values.reshape(-1,1))
titanic_df_impute['RoomService'] = mean_imputer.fit_transform(titanic_df_impute['RoomService'].values.reshape(-1,1))
titanic_df_impute['FoodCourt'] = mean_imputer.fit_transform(titanic_df_impute['FoodCourt'].values.reshape(-1,1))
titanic_df_impute['ShoppingMall'] = mean_imputer.fit_transform(titanic_df_impute['ShoppingMall'].values.reshape(-1,1))
titanic_df_impute['Spa'] = mean_imputer.fit_transform(titanic_df_impute['Spa'].values.reshape(-1,1))
titanic_df_impute['VRDeck'] = mean_imputer.fit_transform(titanic_df_impute['VRDeck'].values.reshape(-1,1))

In [49]:
# Check to see if Mean Imputation was performed
titanic_df_impute.isnull().sum()

HomePlanet      201
CryoSleep       217
Deck            199
RoomNum         199
Side            199
Destination     182
Age               0
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [50]:
## Use ffil and fillna to fill object values with the value nearest to it
titanic_df_impute['RoomNum'] = titanic_df_impute['RoomNum'].fillna(method='ffill')
titanic_df_impute['HomePlanet'] = titanic_df_impute['HomePlanet'].fillna(method='ffill')
titanic_df_impute['CryoSleep'] = titanic_df_impute['CryoSleep'].fillna(method='ffill')
titanic_df_impute['Deck'] = titanic_df_impute['Deck'].fillna(method='ffill')
titanic_df_impute['Side'] = titanic_df_impute['Side'].fillna(method='ffill')
titanic_df_impute['VIP'] = titanic_df_impute['VIP'].fillna(method='ffill')
titanic_df_impute['Destination'] = titanic_df_impute['Destination'].fillna(method='ffill')

In [51]:
# Check to see if categorical Imputation was performed
titanic_df_impute.isnull().sum()

HomePlanet      0
CryoSleep       1
Deck            0
RoomNum         0
Side            0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [52]:
titanic_df_impute

,HomePlanet,CryoSleep,Deck,RoomNum,Side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,None,B,175,S,TRAPPIST-1e,33.0,False,0.0,4017.0,173.729169,311.138778,2260.0,True
1,Earth,False,F,1723,S,55 Cancri e,19.0,False,720.0,2.0,173.729169,311.138778,0.0,False
4,Europa,True,B,1,P,TRAPPIST-1e,34.0,False,0.0,0.0,173.729169,0.000000,0.0,True
5,Mars,False,F,3,P,55 Cancri e,27.0,False,1286.0,122.0,173.729169,0.000000,0.0,False
6,Earth,True,E,0,S,55 Cancri e,29.0,False,0.0,0.0,173.729169,0.000000,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12964,Europa,False,E,315,P,TRAPPIST-1e,32.0,False,8151.0,2052.0,1919.000000,7352.000000,754.0,False
12965,Mars,False,F,465,P,TRAPPIST-1e,37.0,False,20.0,0.0,2559.000000,20.000000,0.0,True
12966,Earth,False,F,739,P,TRAPPIST-1e,20.0,False,0.0,0.0,2687.000000,35.000000,340.0,False
12968,Europa,False,C,257,P,TRAPPIST-1e,41.0,False,0.0,116.0,10705.000000,9181.000000,10.0,False
